In [ ]:
import json
import os

import dotenv
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan

from redbox.models import File, Chunk
from redbox.storage import ElasticsearchStorageHandler

In [ ]:
dotenv.load_dotenv("../.env.test")
# Grab it as a dictionary too for convenience
ENV = dotenv.dotenv_values("../.env.test")


es = Elasticsearch(
    hosts=[
        {
            "host": ENV["ELASTIC_HOST"],
            "port": int(ENV["ELASTIC_PORT"]),
            "scheme": ENV["ELASTIC_SCHEME"],
        }
    ],
    basic_auth=(ENV["ELASTIC_USER"], ENV["ELASTIC_PASSWORD"]),
)

print(json.dumps(es.info().body, indent=4))

In [ ]:
from redbox.storage.elasticsearch import ElasticsearchStorageHandler


elasticsearch_storage_handler = ElasticsearchStorageHandler(
    es_client=es, root_index="redbox-test-data"
)

In [ ]:
test_chunk = Chunk(
    uuid="aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa",
    parent_file_uuid="bbbbbbbb-bbbb-bbbb-bbbb-bbbbbbbbbbbb",
    index=0,
    text="This is a test chunk",
    creator_user_uuid="dev",
    metadata={"test": "test"},
)

In [ ]:
elasticsearch_storage_handler.write_item(test_chunk)

In [ ]:
n_random_chunks = 100000
random_chunks = [
    Chunk(
        parent_file_uuid="cccccccc-cccc-cccc-cccc-cccccccccccc",
        index=i,
        text="This is a test chunk",
        creator_user_uuid="dev",
        metadata={"test": "test"},
    )
    for i in range(n_random_chunks)
]

In [ ]:
# elasticsearch_storage_handler.write_items(random_chunks)

In [ ]:
x = elasticsearch_storage_handler.list_all_items(model_type="chunk")
x

In [ ]:
resp = scan(
    es,
    index="redbox-test-data-chunk",
    query={"query": {"match_all": {}}},
)

resp_unpacked = [x for x in resp]
print(f"Returned {len(resp_unpacked)} items")

In [ ]:
resp_unpacked[0]

In [ ]:
es.delete(index="redbox-test-data-chunk", id="aaaaaaaa-aaaa-aaaa-aaaa-aaaaaaaaaaaa")

In [ ]:
# Delete multiple items

ids_to_delete = elasticsearch_storage_handler.list_all_items(model_type="chunk")

es.delete_by_query(
    index="redbox-test-data-chunk",
    body={"query": {"terms": {"_id": ids_to_delete}}},
)

In [ ]:
elasticsearch_storage_handler.delete_item("bad-uuid", "Chunk")

In [ ]:
elasticsearch_storage_handler.read_item("bad-uuid", "Chunk")

In [ ]:
to_delete = elasticsearch_storage_handler.list_all_items(model_type="Chunk") + [
    "bad-uuid"
]

elasticsearch_storage_handler.delete_items(to_delete, "Chunk")